# Final Visualizations

Import Tools

In [ ]:
!pip install joblib

import pandas as pd
import numpy as np
import joblib
from scipy.stats import chi2_contingency
from time import time
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as ipw
from ipywidgets import HBox, VBox,Label
import matplotlib.pyplot as plt
%matplotlib inline

Load data set

In [ ]:
data = joblib.load ('GroupedAndUngroupedData.pkl')

Function for generating Heat Maps:

In [ ]:
def ShowTwoWayHeatmap (df, row, col, normalize = False, colSort=[], rowSort=[], title=None, xlabel=None, ylabel=None):
    """
    Little function to help with creation of heatmaps.
    @params:
        df                  - required - dataframe containing data to be mapped
        row                 - required - name of column in df to plot on the rows
        col                 - required - name of column in df to plot on the columns
        normalize           - optional - specifies whether the table should be normalized along 'index', 'column', 'all' (or True), or neither (False)
        rowSort             - optional - list specifying how the values in the rows should be sorted
        colSort             - optional - list specifying how the values in the columns should be sorted
        title               - optional - title for graph
        xlabel              - optional - label for x axis
        ylabel              - optional - label for y label
    """
    ctab = pd.crosstab (data[row], data[col], normalize = normalize)
    if rowSort:
        ctab = ctab.loc[rowSort]
    if colSort:
        ctab = ctab[colSort]
    if normalize:
        mult = 100
        fmt = '%.2f%%'
    else:
        mult = 1
        fmt = '%.0f'
    fig = plt.figure()
    fig.set_size_inches(13,6)
    heatmap = plt.pcolor(ctab)
    for y in range (ctab.shape[0]):
        for x in range (ctab.shape[1]):
            plt.text (x + 0.5, y + 0.5, fmt % (ctab.iloc[y,x] * mult),
                       ha='center', va='center')
    plt.yticks(np.arange(len (ctab.index))+0.5, ctab.index)
    plt.xticks(np.arange(len (ctab.columns))+0.5, ctab.columns, rotation=90)
#     plt.colorbar(heatmap, ticks=[0, 0.5, 1]).ax.set_yticklabels(['0', '50%', '100%'])
    if not(title):
        title = f'{row} vs {col}'
    plt.suptitle(title, fontsize=20, fontweight='bold')
    if normalize:
        if (normalize == 'columns') or (normalize == 'index'):
            subtitle = f'Normalized along {normalize}'
        else:
            subtitle = 'Normalized'
        plt.title(subtitle, fontsize=12, horizontalalignment='center')
    if not(xlabel):
        xlabel = col
    plt.xlabel(xlabel, fontsize=14)
    if not (ylabel):
        ylabel = row
    plt.ylabel(ylabel, fontsize=14)
    return fig

In [ ]:
def theme (p):
    """
    Small function to apply common theming options to plotly graphs
    Removes grids and applies a white background
    @params:
        p                  - required - plotly graph object
    """
    p = p.update_xaxes(showgrid=False,title=None)
    p = p.update_yaxes(showgrid=False,title=None)
    p = p.update_layout(plot_bgcolor='rgba(0,0,0,0)')
    return p

# Demographics

### Age Groups by Gender

In [ ]:
x = 'gender_Groups'
groupVar = 'age_Groups'

# First reshape the data so it's in the right format to graph
grouped = data[[x, groupVar]].melt(id_vars=x)
grouped = grouped.groupby([x, 'value']).count()
genderTotals = grouped.groupby('gender_Groups').sum()
grouped['PctOfTotal'] = round (grouped.div(genderTotals, level='gender_Groups') * 100,2)
grouped['PctOfTotal'] = grouped.PctOfTotal.astype(str) + '%'
grouped = grouped.reset_index()
grouped.rename({'value':'age_Groups', 'variable':'Count'}, axis=1, inplace=True)
grouped['age_Groups'] = pd.Categorical(grouped['age_Groups'], ['LateTeens', 'Early20s', 'Mid20s', 'Late20s'])
grouped = grouped.sort_values(['age_Groups','Count'])

# Stacked bar chart showing age group distribution by gender
p = px.bar (grouped, x='gender_Groups', y='Count', color=groupVar, orientation='v', text='PctOfTotal',
            title='Age Distribution by Gender', width=600, height=500)
p.for_each_trace(lambda t: t.update(name=t.name.split('=')[1]))
p.update_traces(textposition='inside', textfont=dict(size=14))
p.update_yaxes(showticklabels=False)
p.update_layout (title_font_size=20)
p.update_xaxes(tickfont=dict(size=20))
theme(p)
p.show()

### Voted

In [ ]:
# Reshape data 
voted = data[['Voted','age']].groupby('Voted').count().reset_index()
voted.rename({'age':'Count'},axis=1,inplace=True)
voted['Voted'] = voted['Voted'].map({0:'No',1:'Yes'})

# Create pie chart
fig = plt.figure()
ax = fig.add_axes([2, 2, 2, 2])
patches, texts, autotexts = ax.pie (voted['Count'], labels=voted['Voted'],autopct='%1.1f%%')
for i in range(0, len(texts)):
    texts[i].set_fontsize(20)
    autotexts[i].set_fontsize(20)
plt.title('Did you vote in the past year?',fontsize=20)
plt.show()

### Ethnicity

In [ ]:
# Group the three least common ehtnicities together under "Other" to avoid excessive labels
data['ethnicity_Groups_Cleaned'] = data.ethnicity_Groups.apply(lambda x: 'Other' if (x=='Latino') or (x=='MiddleEastern') or (x=='OtherEthnicity') else x)

# Reshape data
grouped = pd.DataFrame(data.ethnicity_Groups_Cleaned.value_counts()).reset_index().sort_values('index')
grouped.rename({'index':'ethnicity_Groups_Cleaned', 'ethnicity_Groups_Cleaned':'Count'},axis=1,inplace=True)

# Create pie chart
fig = plt.figure()
fig.set_size_inches (4,4)
ax = fig.add_axes([2, 2, 2, 2])
patches, texts, autotexts = ax.pie (grouped['Count'], labels=grouped['ethnicity_Groups_Cleaned'],autopct='%1.1f%%')
for i in range(0, len(texts)):
    texts[i].set_fontsize(20)
    autotexts[i].set_fontsize(17)
plt.title('Ethnicity Distribution',fontsize=25)
plt.show()

### Area Type

In [ ]:
# Reshape data
x = 'USAAreaType_Groups'
grouped = data[[x]].melt()
grouped = grouped.groupby('value').count()
grouped.rename({'variable':'Count'},axis=1,inplace=True)
grouped['PctOfTotal'] = round (grouped.Count / grouped.Count.sum() * 100, 2)
grouped['PctOfTotal'] = grouped['PctOfTotal'].astype(str) + '%'
grouped = grouped.reset_index()

# Create bar chart
p = px.bar (grouped, x='value', y='Count', text='PctOfTotal', title='Distribution by Area Type',width=600, height=500)
p.update_traces(textposition='outside', textfont=dict(size=14))
p.update_xaxes (type='category', categoryorder='total ascending', tickfont=dict(size=20))
p.update_yaxes (showticklabels=False)
p.update_layout (title_font_size=20)
p = theme(p)
p.show()

### Education Levels

In [ ]:
x = 'USAEducation_Groups'
# Logical sort order
categoryArrayOrder = ['IDK', 'Other', 'NotHighSchoolGrad', 'InHighSchool', 'HighSchoolGrad', 'SomeCollege', '2YearCollege',
                      '4YearCollege', 'Masters', 'Professional(JD/MD)', 'Doctoral']

# Reshape data
grouped = data[[x]].melt()
grouped = grouped.groupby('value').count()
grouped.rename({'variable':'Count'},axis=1,inplace=True)
grouped['PctOfTotal'] = round (grouped.Count / grouped.Count.sum() * 100, 2)
grouped['PctOfTotal'] = grouped['PctOfTotal'].astype(str) + '%'
grouped = grouped.reset_index()

# Create bar chart
p = px.bar (grouped, x='value', y='Count', text='PctOfTotal', title='Education Level')
p.update_traces(textposition='outside', textfont=dict(size=14))
p.update_xaxes(type='category', categoryorder='array', categoryarray=categoryArrayOrder,tickfont=dict(size=15))
p.update_yaxes(showticklabels=False)
p.update_layout (title_font_size=20)
theme(p)
p.show()

### Political Party

In [ ]:
# Clean up the labels and gorup "None" and "IDK" under "N/A"
data['USAPoliticalParty_Clean'] = data.USAPoliticalParty_Groups.map({'Dem':'Democrat','Rep':'Republican', 'Ind':'Independent', 
                                                                     'None':'N/A', 'IDK':'N/A', 'SomethingElse':'Other'})

# Reshape the data
grouped = pd.DataFrame(data.USAPoliticalParty_Clean.value_counts()).reset_index()
grouped.rename ({'index':'Party', 'USAPoliticalParty_Clean':'Count'},axis=1, inplace=True)
grouped['PctOfTotal'] = round (grouped.Count / grouped.Count.sum() * 100, 2)
grouped['PctOfTotal'] = grouped['PctOfTotal'].astype(str) + '%'
grouped['PctText'] = grouped['Party'] + ' - ' + grouped['PctOfTotal']

# Create stacked bar chart
p = px.bar(grouped, y='Count',color='Party', text='PctText', barmode='stack', title='Party Affiliation',
       color_discrete_map={
           'Democrat':'blue',  # Specify the traditional colors for Democrat and Republican (blue & red)
           'Republican':'red',
           'Independent':'#2CA02C'  
       })
p.for_each_trace(lambda t: t.update(name=t.name.split('=')[1]))
p.update_traces(textfont=dict(size=18),textposition='auto', insidetextanchor='middle')
p.update_xaxes(showticklabels=False)
p.update_yaxes(showticklabels=False)
p.update_layout(showlegend=False,title_font_size=20)
p.update_layout(
    title={
        'text': "Party Affiliation",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
theme(p)
p.show()

# Univariate Analysis: Social Media & Video Game Usage

### Social Media Usage - Frequency by platform

In [ ]:
socialMediaCols = ['socialMediaUseFacebook_Groups', 'socialMediaUseTwitter_Groups','socialMediaUseLinkedIn_Groups',
                   'socialMediaUseYouTube_Groups','socialMediaUseInstagram_Groups','socialMediaUsePinterest_Groups',
                   'socialMediaUseMyspace_Groups','socialMediaUseGooglePlus_Groups','socialMediaUseFoursquare_Groups',
                   'socialMediaUseReddit_Groups']

# Logical sort order
frequencyOrder = ['Never','Daily','MultipleTimesPerDay']

# Reshape data
grouped = data[socialMediaCols].melt().reset_index().groupby(['variable','value']).count().reset_index()
grouped.rename({'variable':'Social Media Platform','value':'Frequency','index':'Count'},inplace=True,axis=1)
grouped['Social Media Platform'] = grouped['Social Media Platform'].str.replace('socialMediaUse','')
grouped['Social Media Platform'] = grouped['Social Media Platform'].str.replace('_Groups','')
grouped = grouped[(grouped.Frequency=='Never') | (grouped.Frequency=='MultipleTimesPerDay') | (grouped.Frequency=='Daily')]

# Create grouped bar chart
p = px.bar(grouped, x='Frequency',color='Social Media Platform', y='Count',barmode='group',text='Count')
p.update_traces(textposition='outside', textfont=dict(size=18))
p.for_each_trace(lambda t: t.update(name=t.name.split('=')[1]))
p.update_yaxes(showticklabels=False)
p.update_xaxes(type='category',categoryorder='array',categoryarray=frequencyOrder)
p.update_layout(title_font_size=20,
    title={
        'text': "Social Media Usage - Least and Most Frequent",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
theme(p)
# Remove the text from everything other than the first two markers (Facebook & FourSquare), which are also the most & least
# used, to highlight them.
for i in range (2, len(p.data)):
    p.data[i]['text'] = None
    
p.show()

### Social Media Usage - News Source

In [ ]:
newsSourceCols = {'USAPoliticalNewsCableTV_Groups': 'Cable TV',
 'USAPoliticalNewsSourceRadio_Groups': 'Radio',
 'USAPoliticalNewsSourceDailyNewspapers_Groups': 'Newspapers',
 'USAPoliticalNewsSourceWeeklyMagazines_Groups': 'Magazines',
 'USAPoliticalNewsSourceNetworkTV_Groups': 'Network TV',
 'USAPoliticalNewsSourceLocalTV_Groups': 'Local TV',
 'USAPoliticalNewsSourceLateNightTV_Groups': 'Late Night TV',
 'USAPoliticalNewsSourceComedyTV_Groups': 'Comedy TV',
 'USAPoliticalNewsSourceSocialMedia_Groups': 'Social Media'}

# Reshape data
grouped = data[newsSourceCols.keys()].melt()
grouped.variable = grouped.variable.map(newsSourceCols)
grouped = grouped.reset_index().groupby(['variable','value']).count().reset_index()
grouped = grouped[grouped.value=='pos']
# Make social media a different color to highlight it
grouped['color'] = grouped.variable.apply(lambda x: '1' if x != 'Social Media' else '2')

# Create bar chart
p = px.bar (grouped, x='variable',y='index', text='index', color = 'color')
p.update_xaxes (type='category', categoryorder='total ascending', title = 'Source for Political News')
p.update_yaxes (showticklabels=False)
p.update_traces (textposition='outside', showlegend=False)
p.update_layout(title_font_size=25,showlegend=False,
    title={
        'text': "Source for Political News",
        'y':.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})
theme(p)
p.show()

### Video Game Usage - Frequency

In [ ]:
col = 'videoGameUseFrequency_Groups'
freqSort = ['Never', '<1PerMonth','1PerMonth','2-3PerMonth','1PerWeek','2-3PerWeek','Daily','MultipleTimesPerDay']
grouped = data[[col]].melt().groupby(['value']).count().reset_index()
grouped.rename({'value':'Frequency', 'variable':'Count'},axis=1,inplace=True)

p = px.bar(grouped, x='Frequency', y='Count', text='Count')
p.update_xaxes(type='category',categoryorder='array',categoryarray=freqSort)
p.update_yaxes(showticklabels=False)
p.update_traces(textposition='outside')
theme(p)
p.show()

# Bivariate Analysis: Voted Visualizations 

Figure 1: Video Game Use Frequency vs Voted

In [ ]:
ShowTwoWayHeatmap(data,'videoGameUseFrequency_Groups','Voted', normalize='index',
                  rowSort=["Never", "<1PerMonth", "1PerMonth","2-3PerMonth","1PerWeek","2-3PerWeek","Daily","MultipleTimesPerDay"],
                  title='Video Game Use Frequency vs Voted',
                 ylabel='Video Game Use Frequency')

Figure 2: Learning About Ethical Issues in Video Games vs Voted

In [ ]:
ShowTwoWayHeatmap(data,'videoGamePlayingMoralEthicalIssues_Groups','Voted', normalize='index',
                 title='Learning About Ethical Issues in Video Games vs Voted',
                 ylabel='Learning About Ethical Issues in Video Games')


Figure 3: Facebook Use Frequency vs Voted

In [ ]:
ShowTwoWayHeatmap(data,'socialMediaUseFacebook_Groups','Voted', normalize='index',
                  rowSort=["Never", "<1PerMonth", "1PerMonth","2-3PerMonth","1PerWeek","2-3PerWeek","Daily","MultipleTimesPerDay"],
                  title='Facebook Use Frequency vs Voted',
                 ylabel='Facebook Use Frequency')

# Bivariate Analysis: Political Party Visualizations 

### Political Party by Ethnicity

In [ ]:
color_discrete_map={"Rep": "#EF553B","Ind": "#00CC96","Dem": "#636EFA",}

In [ ]:
partyClean = {'Dem':'Democrat',
              'IDK':'Don''t Know',
              'Ind':'Independent',
              'None': 'None',
              'Rep': 'Republican',
              'SomethingElse': 'Other'}

grouped = data[data.USAWhite==1][['USAPoliticalParty_Groups']]
grouped = grouped.reset_index().groupby('USAPoliticalParty_Groups').count()
grouped.rename ({'index':'Count'},axis=1,inplace=True)
grouped = grouped.reset_index()
grouped.USAPoliticalParty_Groups = grouped.USAPoliticalParty_Groups.map (partyClean)
grouped['Percent'] = round (grouped.Count / sum(grouped.Count) * 100,2).astype(str) + '%'

color_discrete_map={"Republican": "#EF553B","Independent": "#00CC96","Democrat": "#636EFA",}
p=px.bar(grouped, x='USAPoliticalParty_Groups', color='USAPoliticalParty_Groups', y='Count', text='Percent'
         , color_discrete_map=color_discrete_map)
p.update_xaxes (type='category', categoryorder='total descending')
p.update_yaxes (showticklabels=False)
p.for_each_trace(lambda t: t.update(name=t.name.split('=')[1]))
p.update_traces(textposition='outside', textfont=dict(size=14))
p.update_layout(
    title={
        'text': "Political Party of White Respondents",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
theme (p)
p.show()

In [ ]:
fig=px.histogram(data[data['USAWhite']==0], x="USAWhite",color='USAPoliticalParty_Groups',color_discrete_map=color_discrete_map,barnorm='percent',
                labels={
                     "USAWhite": "Ethnicity Non-White",
                     "USAPoliticalParty_Groups": "Political Party",
                     "Ind": "Independent",
                     "Dem": "Democrat",
                     "Rep": "Republican",
                     "IDK": "Don't Know"
                 },
                title="Political Party of Non-White Respondents")
fig.update_layout(barmode='group',yaxis_title="Percent")

### Internet Use

In [ ]:
# These four were ultimately not used
# ShowTwoWayHeatmap(data,'internetUseHealthDifficult_Groups','USAPoliticalParty_Groups', normalize=False)
# ShowTwoWayHeatmap(data,'internetUseVideos_Groups','USAPoliticalParty_Groups', normalize=False)
# ShowTwoWayHeatmap(data,'internetUseBoughtThings_Groups','USAPoliticalParty_Groups', normalize=False)
# ShowTwoWayHeatmap(data,'internetUseTravel_Groups','USAPoliticalParty_Groups', normalize=False)

data['USAPoliticalParty_Groups_Cleaned'] = data.USAPoliticalParty_Groups.map(partyClean)
data['internetUseSocialNetwork_Groups_Cleaned'] = data.internetUseSocialNetwork_Groups.map({'pos':'Yes','neg':'No'}) 

p = ShowTwoWayHeatmap(data,'internetUseSocialNetwork_Groups_Cleaned','USAPoliticalParty_Groups_Cleaned', normalize='columns',
                    title='Use of Social Networks vs Political Parties', xlabel='Political Party', ylabel='Do you use the Internet to access Social Networks?')

### Video Games

In [ ]:
videoGameResponseClean = {'pos':'Yes','neg':'No','NotMe':'Don''t Play Video Games','N/A':'N/A'}
data['videoGamePlayingGamer_Groups_Cleaned'] = data.videoGamePlayingGamer_Groups.map(videoGameResponseClean)
p = ShowTwoWayHeatmap(data,'videoGamePlayingGamer_Groups_Cleaned','USAPoliticalParty_Groups_Cleaned', normalize='columns', 
                  title='Self-Identifying as a Gamer vs Political Party', xlabel=' ', ylabel=' ')

data['videoGamePlayingMoralEthicalIssues_Groups_Cleaned'] = data.videoGamePlayingMoralEthicalIssues_Groups.map(videoGameResponseClean)
p = ShowTwoWayHeatmap(data,'videoGamePlayingMoralEthicalIssues_Groups_Cleaned','USAPoliticalParty_Groups_Cleaned', normalize='columns',
                      title = 'Thinking of Moral/Ethical Issues while Gaming vs Political Party',
                      xlabel=' ', ylabel=' ')


data['videoGamePlayingLearnSocietyProblems_Groups_Cleaned'] = data.videoGamePlayingLearnSocietyProblems_Groups.map(videoGameResponseClean)
p = ShowTwoWayHeatmap(data,'videoGamePlayingLearnSocietyProblems_Groups_Cleaned','USAPoliticalParty_Groups_Cleaned', 
                    normalize='columns',title='Learning about Society\'s Problems while Gaming vs Political Party',
                    xlabel= ' ', ylabel = ' ')

### Self Politics

In [ ]:
internetResponseClean = {'pos':'Yes', 'neutral':'Neutral', 'neg':'No', 'N/A':'N/A'}
data['selfUnderstandPolitics_Groups_Cleaned'] = data.selfUnderstandPolitics_Groups.map(internetResponseClean)
data['selfInternetFindPoliticalInfo_Groups_Cleaned'] = data.selfInternetFindPoliticalInfo_Groups.map(internetResponseClean)

p = ShowTwoWayHeatmap(data,'selfUnderstandPolitics_Groups_Cleaned','USAPoliticalParty_Groups_Cleaned', normalize='columns'
                     ,title='Understanding politics vs Political Party', xlabel = ' ', ylabel = ' ')

p = ShowTwoWayHeatmap(data,'selfInternetFindPoliticalInfo_Groups_Cleaned','USAPoliticalParty_Groups_Cleaned', normalize='columns',
                      title='Find Political Info on Internet vs Political Party', xlabel = ' ', ylabel = ' ')

### Voted

In [ ]:
data['Voted_Cleaned'] = data.Voted.map ({1:'Yes', 0:'No'})

p = ShowTwoWayHeatmap(data,'USAPoliticalParty_Groups_Cleaned','Voted', normalize='index', title='Political Party vs Voted',
                      ylabel = 'Political Party')

### Social Issues

In [ ]:
data['protestEffective_Groups_Cleaned'] = data.protestEffective_Groups.map(internetResponseClean)
p = ShowTwoWayHeatmap(data,'protestEffective_Groups_Cleaned','USAPoliticalParty_Groups_Cleaned', normalize='columns',
                      title='Are protests effective vs Political Party', xlabel=' ', ylabel=' ')

data['govtPoorDependence_Groups_Cleaned'] = data.govtPoorDependence_Groups.map(internetResponseClean)
p = ShowTwoWayHeatmap(data,'govtPoorDependence_Groups_Cleaned','USAPoliticalParty_Groups_Cleaned', normalize='columns',
                      title='Poor Too Dependent on Government vs Political Party', xlabel= ' ', ylabel= ' ')

### News Bias and Facebook

In [ ]:
# These were ultimately not used
# ShowTwoWayHeatmap(data,'newsUnbiased_Groups','USAPoliticalParty_Groups', normalize=False)
# ShowTwoWayHeatmap(data,'facebookEncourageVote_Groups','USAPoliticalParty_Groups', normalize=False)
# ShowTwoWayHeatmap(data,'facebookPostPoliticalLinks_Groups','USAPoliticalParty_Groups', normalize=False)

### News Trustworthy and Government

In [ ]:
# These were ulimtaely not used
# ShowTwoWayHeatmap(data,'newsTrustworthy_Groups','USAPoliticalParty_Groups', normalize=False)
# ShowTwoWayHeatmap(data,'govtBusinessRegulation_Groups','USAPoliticalParty_Groups', normalize=False)

# Bivariate Analysis: Progressivism

### Progressivism - Mean score for govt vs non-govt questions

In [ ]:
score_cols = [col for col in data.columns if '_Score' in col and col != 'progressivism_Score']

# reshape data
meanScores = pd.DataFrame(data[score_cols].agg('mean')).reset_index()
meanScores.rename({'index':'Topic', 0:'Mean Score'}, axis=1, inplace=True)
meanScores['Mean Score'] = round (meanScores['Mean Score'], 2)
# Add column to indicate if question was government-related or not
meanScores['GovtRelated'] = meanScores.Topic.apply(lambda x: 'Government Related' if 'govt' in x else 'Non-Government Related')

colorMap = {'Government Related':'red','Non-Government Related':'blue'}
# Create bar chart
p = px.bar(meanScores, x='Topic', y='Mean Score', color='GovtRelated', text='Mean Score',color_discrete_map = colorMap)
p.for_each_trace(lambda t: t.update(name=t.name.split('=')[1]))
p.update_traces(textposition='outside', textfont=dict(size=18))
p.update_xaxes(showticklabels=False)
p.update_yaxes(showticklabels=False)
theme (p)
p.show()

### Progressivism - by Age Group

In [ ]:
ageSort = ['LateTeens','Early20s','Mid20s','Late20s']

# Reshape data
grouped = data[['age_Groups',*score_cols]].melt(id_vars='age_Groups').groupby(['age_Groups','variable']).sum().reset_index()
grouped.rename({'variable':'Topic', 'value':'Total Score'}, axis =1, inplace=True)
grouped['GovtRelated'] = grouped.Topic.apply(lambda x: 'Government Related' if 'govt' in x else 'Non-Government Related')
grouped = grouped.groupby(['age_Groups','GovtRelated']).sum().reset_index()

# Create bar chart
p= px.bar (grouped, x='age_Groups', y='Total Score', color = 'GovtRelated', barmode='relative', text='Total Score', color_discrete_map=colorMap)
p.for_each_trace(lambda t: t.update(name=t.name.split('=')[1]))
p.update_traces(textposition='outside', textfont=dict(size=18))
p.update_xaxes(tickfont=dict(size=20), type='category', categoryorder='array', categoryarray=ageSort)
p.update_yaxes(showticklabels=False)
p.update_layout(title_font_size=25,
    title={
        'text': "Progressivism Score by Age Group",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
theme (p)
p.show()

### Progressivism vs Facebook & Video Game Usage

In [ ]:
progSort = ['Very Conservative', 'Slightly Conservative', 'Neither', 'Slightly Progressive', 'Very Progressive']
freqSort = ['No', 'Neutral', 'Yes']

# Clean the responses up a bit
facebookDailyRoutineMap = {'1':'No', '2':'No', '3':'Neutral', '4':'Yes', '5':'Yes', '9':'Neutral','':'Neutral'}
data['facebookDailyRoutine_Clean'] = data['facebookDailyRoutine'].map(facebookDailyRoutineMap)
data['facebookDailyRoutine_Clean'] = data['facebookDailyRoutine_Clean'].fillna('Neutral')

# Create heatmap
p = ShowTwoWayHeatmap (data, 'facebookDailyRoutine_Clean', 'progressivism_Groups', normalize='columns',colSort=progSort,
                       rowSort = freqSort, title = 'Daily Facebook Usage vs Progressivism', xlabel= 'Progressivism', ylabel='Daily Facebook Usage')


In [ ]:
row = 'videoGameUseFrequency_Groups'
freqSort = ['Never', '<1PerMonth','1PerMonth','2-3PerMonth','1PerWeek','2-3PerWeek','Daily','MultipleTimesPerDay']
col = 'progressivism_Groups'

p = ShowTwoWayHeatmap (data,row,col,normalize='index', rowSort = freqSort, colSort = progSort,
                      title = 'Video Game Usage vs Progressivism', xlabel= 'Progressivism', ylabel='Video Game Usage')